In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics, linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMRegressor
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

path = "/home/darragh/avito/data/"
path = '/Users/dhanley2/Documents/avito/data/'
#path = '/home/ubuntu/avito/data/'
#data_path = 'data/'

In [2]:
#lgb25 = pd.read_csv('../lgCV_2505.csv.gz', compression='gzip')
lgb05 = pd.read_csv(path + '../sub/lgCV_0506.csv.gz', compression='gzip')
lgb02A = pd.read_csv(path + '../sub/lgCV_0206A.csv.gz', compression='gzip')
lgb27 = pd.read_csv(path  + '../sub/lgCV_2705B.csv.gz', compression='gzip')
lgb31 = pd.read_csv(path  + '../sub/lgCV_3105.csv.gz', compression='gzip')
lgb02 = pd.read_csv(path  + '../sub/lgCV_0206.csv.gz', compression='gzip')
rnn =   pd.read_csv(path  + '../sub/rnnCV_2805.csv.gz', compression='gzip')
rnn27 = pd.read_csv(path  + '../sub/rnnCV_2705A.csv.gz', compression='gzip')
rnn05 = pd.read_csv(path  + '../sub/rnnCV_0506.csv.gz', compression='gzip')

mlp =   pd.read_csv(path  + '../sub/mlpCV_2505.csv.gz', compression='gzip')
rdgimg =   pd.read_csv(path  + '../features/ridgeImg5CV.csv.gz', compression='gzip')
truth = pd.read_csv(path+'train.csv.zip', compression='zip', parse_dates = ["activation_date"])
y =     truth['deal_probability'].values
truth.drop('deal_probability', 1)
test =  pd.read_csv(path+'test.csv.zip', compression='zip', parse_dates = ["activation_date"])
test['deal_probability']=float('NAN') 
truth = pd.concat([truth,test[truth.columns]],axis=0)

In [3]:
rdgimg['item_id'] = truth['item_id'].values
rdgimg.head(2)

,ridge_img_preds,item_id
0,-0.023149,b912c3c6a6ad
1,0.222946,2dac0150717d


In [6]:
#lgb25.rename(columns={'deal_probability' : 'lgb25_preds' }, inplace=True)
lgb31.rename(columns={'deal_probability'  : 'lgb31_preds' }, inplace=True)
lgb27.rename(columns={'deal_probability'  : 'lgb27_preds' }, inplace=True)
lgb02.rename(columns={'deal_probability'  : 'lgb02_preds' }, inplace=True)
lgb02A.rename(columns={'deal_probability' : 'lgb02A_preds' }, inplace=True)
rnn27.rename(columns={'deal_probability'  : 'rnn27_preds' }, inplace=True)
rnn05.rename(columns={'deal_probability'  : 'rnn05_preds' }, inplace=True)
mlp.rename(columns={'deal_probability'    : 'mlp_preds' }, inplace=True)
lgb05.rename(columns={'deal_probability'  : 'lgb05_preds' }, inplace=True)
rdgimg.rename(columns={'deal_probability'  : 'rdgim_preds' }, inplace=True)
preds_df = lgb27.merge(rnn, on='item_id')\
                .merge(mlp, on='item_id')\
                .merge(lgb31, on='item_id')\
                .merge(lgb02, on='item_id')\
                .merge(lgb05, on='item_id')\
                .merge(lgb02A, on='item_id')\
                .merge(rnn27, on='item_id')\
                .merge(rnn05, on='item_id')\
                .merge(rdgimg, on='item_id')\
                .merge(truth, on='item_id',how='left')

In [7]:
preds_df.columns

Index(['item_id', 'lgb27_preds', 'rnn_preds', 'mlp_preds', 'lgb31_preds',
       'lgb02_preds', 'lgb05_preds', 'lgb02A_preds', 'Unnamed: 0',
       'rnn27_preds', 'rnn05_preds', 'ridge_img_preds', 'user_id', 'region',
       'city', 'parent_category_name', 'category_name', 'param_1', 'param_2',
       'param_3', 'title', 'description', 'price', 'item_seq_number',
       'activation_date', 'user_type', 'image', 'image_top_1',
       'deal_probability'],
      dtype='object')

In [8]:
preds_df['difference_l02_l05']=preds_df['lgb02A_preds'] - preds_df['lgb05_preds']
preds_df['difference_l02_r']=preds_df['lgb02A_preds'] - preds_df['rnn_preds']
preds_df['difference_l02_r1']=preds_df['lgb02_preds'] - preds_df['rnn27_preds']
preds_df['difference_l02_r05']=preds_df['lgb02_preds'] - preds_df['rnn05_preds']
preds_df['difference_l02_m']=preds_df['lgb02_preds'] - preds_df['mlp_preds']
preds_df['difference_r_m']=preds_df['rnn27_preds'] - preds_df['mlp_preds']
preds_df['difference_r05_m']=preds_df['rnn05_preds'] - preds_df['mlp_preds']
preds_df['difference_l02_l02A']=preds_df['lgb02_preds'] - preds_df['lgb02A_preds']
preds_df['difference_l02_rdgimg']=preds_df['lgb02_preds'] - preds_df['ridge_img_preds']

#preds_df['difference_l27_l25']=preds_df['lgb27_preds'] - preds_df['lgb25_preds']
preds_df['price'].fillna(-1,inplace=True)

preds_df['max'] = np.max(np.array([preds_df[col] for col in preds_df.columns if '_preds' in col]),axis=0)
preds_df['min'] = np.min(np.array([preds_df[col] for col in preds_df.columns if '_preds' in col]),axis=0)
preds_df['avg'] = np.mean(np.array([preds_df[col] for col in preds_df.columns if '_preds' in col]),axis=0)
preds_df['std'] = np.std(np.array([preds_df[col] for col in preds_df.columns if '_preds' in col]),axis=0)

In [9]:
preds_df.head()

,item_id,lgb27_preds,rnn_preds,mlp_preds,lgb31_preds,lgb02_preds,lgb05_preds,lgb02A_preds,Unnamed: 0,rnn27_preds,...,difference_l02_r05,difference_l02_m,difference_r_m,difference_r05_m,difference_l02_l02A,difference_l02_rdgimg,max,min,avg,std
0,b912c3c6a6ad,0.091656,0.069891,0.091223,0.083219,0.090855,0.331958,0.109617,0,0.068331,...,0.026845,-0.000368,-0.022892,-0.027213,-0.018762,0.114004,0.331958,-0.023149,0.097761,0.085282
1,2dac0150717d,0.147801,0.075968,0.211844,0.157973,0.119609,0.350151,0.160045,1,0.070638,...,0.031387,-0.092236,-0.141206,-0.123623,-0.040436,-0.103337,0.350151,0.070638,0.160520,0.080491
2,ba83aefab5dc,0.186261,0.167167,0.236579,0.227048,0.258980,0.396762,0.241336,2,0.161924,...,0.084265,0.022401,-0.074655,-0.061864,0.017644,0.141993,0.396762,0.116987,0.216776,0.073079
3,02996f1dd2ea,0.241699,0.260876,0.383699,0.231447,0.284130,0.530174,0.336456,3,0.278804,...,-0.001151,-0.099569,-0.104895,-0.098417,-0.052325,0.233067,0.530174,0.051063,0.288363,0.115099
4,7c90be56d2ab,0.407423,0.431520,0.420657,0.412563,0.430581,0.879136,0.421989,4,0.350284,...,0.040791,0.009924,-0.070373,-0.030867,0.008593,0.253959,0.879136,0.176622,0.432056,0.165777


### Train scores

In [10]:
idx = preds_df['deal_probability']==preds_df['deal_probability']
print(idx.value_counts())
for col in [c for c in preds_df.columns if '_preds' in c]:
    print('RMSE %s: '%(col), np.sqrt(metrics.mean_squared_error(preds_df['deal_probability'][idx].values, preds_df[col][idx].values)))

True     1503424
False     508438
Name: deal_probability, dtype: int64
RMSE lgb27_preds:  0.21681527582821059
RMSE rnn_preds:  0.21771476573780568
RMSE mlp_preds:  0.2187544351068867
RMSE lgb31_preds:  0.21681135578306754
RMSE lgb02_preds:  0.21625386267187377
RMSE lgb05_preds:  0.35764962408684847
RMSE lgb02A_preds:  0.21627136604335995
RMSE rnn27_preds:  0.21698040871120608
RMSE rnn05_preds:  0.21609044829938392
RMSE ridge_img_preds:  0.2515771855276296


In [11]:
foldls = [["2017-03-15", "2017-03-16", "2017-03-17"], \
       ["2017-03-18", "2017-03-19", "2017-03-20"], \
       ["2017-03-21", "2017-03-22", "2017-03-23"], \
       ["2017-03-24", "2017-03-25", "2017-03-26"], \
        ["2017-03-27", "2017-03-28", "2017-03-29", \
            "2017-03-30", "2017-03-31", "2017-04-01", \
            "2017-04-02", "2017-04-03","2017-04-07"]]
foldls = [[pd.to_datetime(d) for d in f] for f in foldls]
preds_df['fold'] = -1
for t, fold in enumerate(foldls):
    preds_df['fold'][preds_df.activation_date.isin(fold)] = t

In [12]:
scores = []
for col in [c for c in preds_df.columns if '_preds' in c]:
    lstmp = [col]
    for i in range(5):
        idx = preds_df['fold']==i
        lstmp.append(np.sqrt(metrics.mean_squared_error(preds_df['deal_probability'][idx].values, \
                                                        preds_df[col][idx].values)))
    scores.append(lstmp)
pd.DataFrame(scores, columns = ['Model']+['Fold%s'%(i) for i in range(5)])

,Model,Fold0,Fold1,Fold2,Fold3,Fold4
0,lgb27_preds,0.216848,0.217096,0.217923,0.217597,0.213692
1,rnn_preds,0.218756,0.218966,0.215764,0.219554,0.214747
2,mlp_preds,0.219660,0.220088,0.216672,0.220347,0.216385
3,lgb31_preds,0.216537,0.216643,0.219230,0.217169,0.213430
4,lgb02_preds,0.216371,0.216447,0.217354,0.217032,0.213158
5,lgb05_preds,0.350266,0.344368,0.358510,0.357346,0.384583
6,lgb02A_preds,0.216081,0.216051,0.218825,0.216562,0.212741
7,rnn27_preds,0.218097,0.218115,0.214982,0.218651,0.214379
8,rnn05_preds,0.217164,0.217243,0.214235,0.217797,0.213266
9,ridge_img_preds,0.251945,0.252174,0.250096,0.253229,0.250088


### Correlations in test and train

In [13]:
# Test correlation
preds_df[~preds_df['deal_probability'].isnull()][[c for c in preds_df.columns if '_preds' in c]].corr()

,lgb27_preds,rnn_preds,mlp_preds,lgb31_preds,lgb02_preds,lgb05_preds,lgb02A_preds,rnn27_preds,rnn05_preds,ridge_img_preds
lgb27_preds,1.000000,0.915952,0.919171,0.990750,0.988682,0.790839,0.983143,0.916804,0.929808,0.461679
rnn_preds,0.915952,1.000000,0.929710,0.910602,0.913307,0.748368,0.911443,0.985360,0.969063,0.406500
mlp_preds,0.919171,0.929710,1.000000,0.914307,0.916343,0.777301,0.913600,0.925010,0.921354,0.429014
lgb31_preds,0.990750,0.910602,0.914307,1.000000,0.991984,0.789400,0.986851,0.914204,0.927547,0.460846
lgb02_preds,0.988682,0.913307,0.916343,0.991984,1.000000,0.789968,0.987434,0.916726,0.929946,0.458602
lgb05_preds,0.790839,0.748368,0.777301,0.789400,0.789968,1.000000,0.784506,0.750656,0.755479,0.540296
lgb02A_preds,0.983143,0.911443,0.913600,0.986851,0.987434,0.784506,1.000000,0.914911,0.927837,0.455175
rnn27_preds,0.916804,0.985360,0.925010,0.914204,0.916726,0.750656,0.914911,1.000000,0.976853,0.428697
rnn05_preds,0.929808,0.969063,0.921354,0.927547,0.929946,0.755479,0.927837,0.976853,1.000000,0.430368
ridge_img_preds,0.461679,0.406500,0.429014,0.460846,0.458602,0.540296,0.455175,0.428697,0.430368,1.000000


In [14]:
# Train correlation
preds_df[preds_df['deal_probability'].isnull()][[c for c in preds_df.columns if '_preds' in c]].corr()

,lgb27_preds,rnn_preds,mlp_preds,lgb31_preds,lgb02_preds,lgb05_preds,lgb02A_preds,rnn27_preds,rnn05_preds,ridge_img_preds
lgb27_preds,1.000000,0.930288,0.924283,0.991439,0.989502,0.767850,0.984742,0.930692,0.935052,0.448051
rnn_preds,0.930288,1.000000,0.939951,0.927835,0.927867,0.740072,0.928496,0.988198,0.958934,0.403428
mlp_preds,0.924283,0.939951,1.000000,0.922413,0.921927,0.753849,0.922121,0.935069,0.923387,0.408743
lgb31_preds,0.991439,0.927835,0.922413,1.000000,0.993168,0.768690,0.988043,0.931468,0.935803,0.448168
lgb02_preds,0.989502,0.927867,0.921927,0.993168,1.000000,0.766264,0.989577,0.931447,0.934627,0.445530
lgb05_preds,0.767850,0.740072,0.753849,0.768690,0.766264,1.000000,0.764251,0.739462,0.744216,0.533338
lgb02A_preds,0.984742,0.928496,0.922121,0.988043,0.989577,0.764251,1.000000,0.932071,0.934155,0.443897
rnn27_preds,0.930692,0.988198,0.935069,0.931468,0.931447,0.739462,0.932071,1.000000,0.967277,0.424131
rnn05_preds,0.935052,0.958934,0.923387,0.935803,0.934627,0.744216,0.934155,0.967277,1.000000,0.425905
ridge_img_preds,0.448051,0.403428,0.408743,0.448168,0.445530,0.533338,0.443897,0.424131,0.425905,1.000000


In [15]:


cols = [c for c in preds_df.columns if '_preds' in c]
cols += [c for c in preds_df.columns if 'difference' in c]
cols += ['price', 'max', 'min', 'avg', 'std']
categories = ['region', 'param_1', 'parent_category_name', 'category_name', 'param_2' ]
cols += categories
for col in categories:
    preds_df[col] = LabelEncoder().fit_transform(preds_df[col].fillna("0"))
train_df = preds_df[~preds_df['deal_probability'].isnull()]
test_df = preds_df[preds_df['deal_probability'].isnull()]

In [16]:
n_estimators = 4000
train_X, valid_X, train_y, valid_y = train_test_split(train_df[cols], y, train_size=.8, random_state=12345)
eval_set = [(train_X,train_y),(valid_X,valid_y)]

In [17]:
len(train_X)

1202739

In [18]:
%%time
clf = LGBMRegressor(n_estimators=n_estimators, 
                    max_depth=-1, 
                    feature_fraction= 0.5,
                    num_leaves=32, 
                    learning_rate=.01)#, device='gpu')
clf.fit(train_X, train_y, early_stopping_rounds=80, 
        eval_set=eval_set, eval_metric='rmse', verbose=100, 
        categorical_feature=categories)
# valid_1's rmse: 0.213771

Training until validation scores don't improve for 80 rounds.
[100]	training's rmse: 0.220606	valid_1's rmse: 0.220957
[200]	training's rmse: 0.214262	valid_1's rmse: 0.214851
[300]	training's rmse: 0.213102	valid_1's rmse: 0.213896
[400]	training's rmse: 0.212753	valid_1's rmse: 0.213712
[500]	training's rmse: 0.212547	valid_1's rmse: 0.213663
[600]	training's rmse: 0.212381	valid_1's rmse: 0.213634
[700]	training's rmse: 0.212226	valid_1's rmse: 0.213612
[800]	training's rmse: 0.21208	valid_1's rmse: 0.213595
[900]	training's rmse: 0.211936	valid_1's rmse: 0.213581
[1000]	training's rmse: 0.211802	valid_1's rmse: 0.21357
[1100]	training's rmse: 0.211674	valid_1's rmse: 0.213562
[1200]	training's rmse: 0.211551	valid_1's rmse: 0.213555
[1300]	training's rmse: 0.211431	valid_1's rmse: 0.213547
[1400]	training's rmse: 0.211316	valid_1's rmse: 0.213542
[1500]	training's rmse: 0.211206	valid_1's rmse: 0.213538
[1600]	training's rmse: 0.211094	valid_1's rmse: 0.213535
[1700]	training's rms

In [19]:
sorted(zip(clf.feature_importances_, train_X.columns ),key=lambda x: -x[0])

[(9875, 'param_1'),
 (6195, 'param_2'),
 (5689, 'region'),
 (2814, 'category_name'),
 (2515, 'mlp_preds'),
 (2300, 'rnn05_preds'),
 (2193, 'rnn27_preds'),
 (2160, 'difference_l02_l05'),
 (2141, 'lgb02A_preds'),
 (2019, 'difference_l02_l02A'),
 (2017, 'difference_r_m'),
 (1995, 'difference_l02_r'),
 (1967, 'difference_l02_r05'),
 (1946, 'difference_l02_r1'),
 (1924, 'difference_r05_m'),
 (1894, 'difference_l02_m'),
 (1809, 'ridge_img_preds'),
 (1710, 'min'),
 (1639, 'std'),
 (1592, 'price'),
 (1518, 'avg'),
 (1517, 'rnn_preds'),
 (1477, 'lgb02_preds'),
 (1440, 'lgb05_preds'),
 (1309, 'lgb27_preds'),
 (1242, 'max'),
 (1231, 'lgb31_preds'),
 (1206, 'difference_l02_rdgimg'),
 (401, 'parent_category_name')]

In [21]:
n_estimators = 2185
train_X = train_df[cols]
train_y = y
eval_set = [(train_X,train_y)]
len(train_X)

1503424

In [22]:
%%time
clf = LGBMRegressor(n_estimators=n_estimators, 
                    max_depth=-1, 
                    feature_fraction= 0.5,
                    num_leaves=32, 
                    learning_rate=.01)#, device='gpu')

clf.fit(train_X, train_y, early_stopping_rounds=80, 
        eval_set=eval_set, eval_metric='rmse', verbose=100, 
        categorical_feature=categories)

Training until validation scores don't improve for 80 rounds.
[100]	training's rmse: 0.220683
[200]	training's rmse: 0.214381
[300]	training's rmse: 0.213251
[400]	training's rmse: 0.212926
[500]	training's rmse: 0.212739
[600]	training's rmse: 0.212588
[700]	training's rmse: 0.21245
[800]	training's rmse: 0.212327
[900]	training's rmse: 0.212212
[1000]	training's rmse: 0.212103
[1100]	training's rmse: 0.211993
[1200]	training's rmse: 0.211894
[1300]	training's rmse: 0.211794
[1400]	training's rmse: 0.211697
[1500]	training's rmse: 0.211603
[1600]	training's rmse: 0.21151
[1700]	training's rmse: 0.211421
[1800]	training's rmse: 0.211329
[1900]	training's rmse: 0.211238
[2000]	training's rmse: 0.21115
[2100]	training's rmse: 0.211059
Did not meet early stopping. Best iteration is:
[2185]	training's rmse: 0.210986
CPU times: user 11min 11s, sys: 39.4 s, total: 11min 50s
Wall time: 2min 26s


In [23]:
test_df['deal_probability'] = clf.predict(test_df[cols])
test_df['deal_probability'] = np.clip(test_df['deal_probability'], .0001, .9999)
test_df[['item_id', 'deal_probability']].to_csv('../lgbbsub_0806L2.csv.gz', compression='gzip', index=False, header=True)

In [24]:
test_df[['item_id', 'deal_probability']].head()

,item_id,deal_probability
1503424,6544e41a8817,0.429742
1503425,65b9484d670f,0.161334
1503426,8bab230b2ecd,0.122211
1503427,8e348601fefc,0.062077
1503428,8bd2fe400b89,0.200186
